<a href="https://colab.research.google.com/github/seulmi0827/fininsight/blob/main/JACE/BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q bertopic
!pip install -q bertopic[visualization]

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk -y
!pip install konlpy
!pip install mecab-python
!apt-get install curl -y
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 전처리

In [ ]:
from datetime import datetime

def parse_datetime(date_string):
   try:
       return datetime.fromisoformat(date_string)
   except ValueError:
       print(f"Invalid date format: {date_string}")
       return None


from konlpy.tag import Mecab
import pandas as pd

mecab = Mecab()

def preprocess(text):
    pos_tagged = mecab.pos(text)

    filtered = [
        word for word, pos in pos_tagged
        if (
            pos.startswith('NN') or  # 명사
            pos.startswith('VV') or  # 동사
            pos.startswith('VA') or  # 형용사
            pos == 'MAG'             # 일반 부사
        ) and len(word) > 1          # 1글자 이상만
    ]

    return filtered


def load_and_preprocess_data(file_path):
    # 데이터프레임 로드
    df = pd.read_csv(file_path)

    # 주말 뉴스 비율 확인 (참고용)
    test = pd.read_csv(file_path)
    test['inp_date'] = pd.to_datetime(test['inp_date'])
    test['weekday'] = test['inp_date'].dt.weekday

    # 날짜 범위 계산
    min_date = test['inp_date'].min()
    max_date = test['inp_date'].max()
    total_days = (max_date - min_date).days + 1

    # 주말 날짜 수 계산
    date_range = pd.date_range(start=min_date, end=max_date)
    weekend_days = sum(date.weekday() >= 5 for date in date_range)
    weekday_days = total_days - weekend_days

    print(f"데이터 기간: {min_date} ~ {max_date}")
    print(f"총 기간: {total_days}일")
    print(f"평일 기간: {weekday_days}일")
    print(f"주말 기간: {weekend_days}일")

    # 주말 날짜 확인
    print("주말 날짜 확인 : ")
    print(test[test['weekday'].isin([5, 6])][['inp_date', 'weekday']])
    print()

    # 통계 출력
    total_news = len(test)
    weekend = len(test[test['weekday'].isin([5, 6])])
    weekday = total_news - weekend

    print(f"평일 기사 수: {weekday}개 ({weekday / total_news * 100:.2f}%)")
    print(f"주말 기사 수: {weekend}개 ({weekend / total_news * 100:.2f}%)")

    # 본 데이터 전처리
    df['inp_date'] = df['inp_date'].apply(parse_datetime)
    df = df[~df['inp_date'].dt.dayofweek.isin([5, 6])]
    print('주말 제거된 문서 수 : ', len(df))

    # 주말 제거 후 날짜별 데이터 확인
    df['date'] = df['inp_date'].dt.date  # 시간 정보 제외한 날짜만 추출
    date_counts = df['date'].value_counts().sort_index()

    # 전체 기간 생성
    full_date_range = pd.date_range(start=df['inp_date'].min().date(),
                                   end=df['inp_date'].max().date(),
                                   freq='D')

    # 주말 제외한 평일만 필터링
    weekday_dates = [date for date in full_date_range if date.weekday() < 5]

    # 날짜별 문서 수 통계
    print("\n== 날짜별 문서 수 통계 ==")
    print(f"평균 문서 수: {date_counts.mean():.2f}")
    print(f"최소 문서 수: {date_counts.min()} (날짜: {date_counts.idxmin()})")
    print(f"최대 문서 수: {date_counts.max()} (날짜: {date_counts.idxmax()})")

    # 텍스트 전처리
    df['content'] = df['content'].fillna('').astype(str)
    df['preprocessed_content'] = df['content'].apply(lambda x: ' '.join(preprocess(x)))

    # 토픽 모델링용 데이터 준비
    preprocessed_content = df['content'].apply(preprocess).tolist()

    return df, preprocessed_content

In [ ]:
df, preprocessed_content = load_and_preprocess_data("/content/drive/MyDrive/Colab Notebooks/삼성전자_10000.csv")

In [ ]:
print('새로 추가된 전처리 컬럼 확인 : ', df.columns.tolist())
print()
print(df[['inp_date', 'preprocessed_content']][:20])

# Mecab과 SBERT를 이용한 Bertopic

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN

import torch
import numpy as np

import random

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def initialize_topic_model(seed):
    umap_model = UMAP(
        n_neighbors=15,
        n_components=5,
        min_dist=0.0,
        metric='cosine',
        random_state=seed
    )

    hdbscan_model = HDBSCAN(
        min_cluster_size=20,
        metric='euclidean',
        prediction_data=True,
        gen_min_span_tree=True,
        cluster_selection_method='eom'
    )

    embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
    vectorizer = CountVectorizer(stop_words=None)

    return BERTopic(
        language="korean",
        nr_topics=20, # 유효 토픽 결정
        top_n_words=7, # 토픽 별 키워드 수 결정
        calculate_probabilities=True,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        embedding_model=embedding_model,
        vectorizer_model=vectorizer,
        verbose=True
    )

def run_topic_modeling(preprocessed_content):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("사용중인 디바이스 : ", device)
    print()

    # SEED = 42
    # set_seed(SEED)
    random_seed = random.randint(0, 10000)

    topic_model = initialize_topic_model(random_seed)
    content_for_topic = [' '.join(doc) for doc in preprocessed_content]
    topics, probs = topic_model.fit_transform(content_for_topic)

    return topics, probs, topic_model

In [ ]:
topics, probs, topic_model = run_topic_modeling(preprocessed_content)

In [ ]:
def analyze_topic_distribution(topic_model, topics):
    # 토픽 정보 가져오기
    topic_info = topic_model.get_topic_info()

    # 유효 토픽과 노이즈 분리
    valid_topics = topic_info[topic_info['Topic'] != -1]
    noise_info = topic_info[topic_info['Topic'] == -1]

    # 토픽 통계 계산
    num_topics = valid_topics.shape[0]
    total_docs = len(topics)
    noise_count = noise_info['Count'].values[0] if not noise_info.empty else 0
    valid_docs = total_docs - noise_count

    # 결과 출력
    print(f"=== 토픽 분석 결과 ===")
    print(f"유효 토픽 수: {num_topics}개")
    print(f"전체 문서 수: {total_docs}개")
    print(f"노이즈 문서 수: {noise_count}개 ({noise_count/total_docs:.2%})")
    print(f"유효 토픽 문서 수: {valid_docs}개 ({valid_docs/total_docs:.2%})")
    print()

    # 아웃라이너 토픽의 키워드 확인
    print("아웃라이너 토픽(-1)의 대표 키워드:")
    try:
        outlier_words = topic_model.get_topic(-1)
        if outlier_words:
            # 상위 10개 키워드 출력
            for word, score in outlier_words[:10]:
                print(f"  - {word} ({score:.4f})")
        else:
            print("  키워드가 없습니다.")
    except:
        print("  아웃라이너 토픽의 키워드를 가져올 수 없습니다.")
    print()

    # 토픽별 문서 분포 (아웃라이너 포함)
    print("토픽별 문서 분포:")
    result_df = topic_info[['Topic', 'Count', 'Representation']].copy()
    result_df['              비율(%)'] = (result_df['Count'] / total_docs * 100).round(2)
    print(result_df)

    return valid_topics, noise_count

In [ ]:
valid_topics, noise_count = analyze_topic_distribution(topic_model, topics)

# 시각화 주말 제외 일 단위 : 실패

In [ ]:
def visualize_topics_over_time(df, topic_model, topics):
    # 날짜 데이터 준비
    timestamps = df['inp_date'].tolist()

    # 시계열 토픽 시각화
    topics_over_time = topic_model.topics_over_time(
        df['preprocessed_content'].tolist(),
        timestamps,
        topics=topics,
        nr_bins=59,  # 평일 기간에 맞춤
        datetime_format="%Y-%m-%d %H:%M:%S"
    )

    # 일 단위 분석 증명을 위한 출력
    print("== 시계열 분석 정보 ==")
    print(f"시계열 데이터 형태: {topics_over_time.shape}")
    print(f"시계열 데이터 컬럼: {topics_over_time.columns.tolist()}")
    print("\n== 날짜 샘플 (처음 5개) ==")
    unique_timestamps = topics_over_time['Timestamp'].unique()
    print(f"총 타임스탬프 수: {len(unique_timestamps)}")
    print(unique_timestamps[:5])

    # 시계열 시각화 - 툴팁에 날짜 포맷 추가
    fig = topic_model.visualize_topics_over_time(
        topics_over_time,
        top_n_topics=10,
        width=1200,
        height=600
    )

    # 툴팁 날짜 형식 수정
    fig.update_traces(
        hovertemplate='<b>토픽: %{customdata}</b><br>날짜: %{x|%Y-%m-%d}<br>빈도: %{y:.2f}<extra></extra>'
    )

    # X축 날짜 형식 수정
    fig.update_xaxes(
        tickformat="%Y-%m-%d",
        title="날짜"
    )

    # Y축 레이블 수정
    fig.update_yaxes(
        title="토픽 빈도"
    )

    # 제목 추가
    fig.update_layout(
        title="시간에 따른 토픽 변화",
        legend_title="토픽"
    )

    return fig, topics_over_time

In [ ]:
# 82일 설정
fig, topics_over_time = visualize_topics_over_time(df, topic_model, topics)
fig.show()

##

전처리 과정에서 주말을 제외했는데도 topic_model.topics_over_time의 시각화에서 주말이 포함되는 것은 이 함수가 데이터의 시간 범위를 균등하게 나누는 방식 때문입니다.
topic_model.topics_over_time 함수는 시작 날짜와 끝 날짜 사이를 nr_bins 개수만큼 균등하게 나누어 시간 구간을 생성합니다. 이 과정에서 주말/평일 구분은 하지 않습니다. 따라서 전체 기간(1월 12일~4월 3일)을 nr_bins=59개로 나누면, 각 구간에는 주말이 포함된 날짜가 표시될 수 있습니다.

topic_model.topics_over_time은 사용하지 않는 걸로 결론

# 일단위+주단위

In [ ]:
def create_weekly_topic_distribution(df, topics, topic_model):
    """주 단위 토픽 분포를 계산하는 함수"""
    # 토픽 정보 가져오기
    topic_df = topic_model.get_topic_info()

    # 리스트 형태의 키워드를 문자열로 변환
    topic_df['Representation'] = topic_df['Representation'].apply(lambda x: ', '.join(x))

    # 토픽 ID와 키워드를 원본 데이터프레임에 연결
    df_with_topics = df.copy()
    df_with_topics['topic_id'] = topics
    df_with_topics['topic_keywords'] = df_with_topics['topic_id'].map(
        dict(zip(topic_df['Topic'], topic_df['Representation']))
    )

    # 주 단위 시간 생성 (시작일은 월요일)
    df_with_topics['time_unit'] = df_with_topics['inp_date'].dt.to_period('W').apply(lambda r: r.start_time)

    # 주 단위로 토픽 카운트 집계
    topic_keyword_distribution = df_with_topics.groupby(['time_unit', 'topic_keywords']).size().unstack(fill_value=0)

    # 총 주 수 및 주 시작일 출력
    num_weeks = len(topic_keyword_distribution.index)
    print(f"총 주 수: {num_weeks}주")
    print("주 시작일 목록:")
    for week_start in topic_keyword_distribution.index:
        print(week_start.strftime("%Y-%m-%d"))

    return topic_keyword_distribution

In [ ]:
weekly_distribution = create_weekly_topic_distribution(df, topics, topic_model)

In [ ]:
def create_topic_timeseries(df, topics, topic_model, time_unit='day', custom_data=None):

    import plotly.graph_objs as go
    import pandas as pd

    # 토픽 정보 가져오기
    topic_info = topic_model.get_topic_info()

    # 아웃라이어 제외한 모든 토픽(-1 제외)
    all_topics = topic_info[topic_info['Topic'] != -1]['Topic'].tolist()

    # 색상 팔레트
    colors = [
        '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
        '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
        '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5',
        '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d', '#9edae5',
        '#636363', '#6baed6', '#fd8d3c', '#74c476', '#969696',
        '#3182bd', '#e6550d', '#31a354', '#756bb1', '#de2d26'
    ]

    # Figure 생성
    fig = go.Figure()

    if time_unit == 'day':
        # 일별 분석
        topic_df = pd.DataFrame({
            'date': df['inp_date'].dt.date,
            'topic': topics
        })

        # 일별-토픽별 집계
        daily_counts = topic_df.groupby(['date', 'topic']).size().reset_index(name='count')

        # 각 토픽별 trace 추가
        for i, topic_id in enumerate(all_topics):
            topic_data = daily_counts[daily_counts['topic'] == topic_id]

            # 토픽 단어 가져오기
            topic_words = [word for word, _ in topic_model.get_topic(topic_id)][:3]
            topic_label = f"토픽 {topic_id}: {', '.join(topic_words)}"

            fig.add_trace(go.Scatter(
                x=topic_data['date'],
                y=topic_data['count'],
                mode='lines+markers',
                name=topic_label,
                line=dict(color=colors[i % len(colors)]),
                marker=dict(size=6),
                hovertemplate='<b>%{text}</b><br>날짜: %{x|%Y-%m-%d}<br>문서 수: %{y}<extra></extra>',
                text=[topic_label] * len(topic_data)
            ))

        # 레이아웃 설정
        fig.update_layout(
            title={
                'text': '삼성전자 뉴스 키워드 분석<br><sup>총기간 2025-01-12 ~ 2025-04-03 : 전체82일 - 주말23일 = 평일59일 일 단위</sup>',
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {'size': 20, 'color': '#1f1f1f'}
            },
            xaxis=dict(title='날짜', tickformat='%Y-%m-%d', gridcolor='lightgray'),
            yaxis=dict(title='문서 수', gridcolor='lightgray'),
            legend=dict(title='토픽', orientation='v'),
            hovermode='closest',
            plot_bgcolor='white'
        )

    elif time_unit == 'week':
        # 주별 분석
        if custom_data is None:
            # 토픽 정보 가져오기
            topic_df = topic_model.get_topic_info()

            # 리스트 형태의 키워드를 문자열로 변환
            topic_df['Representation'] = topic_df['Representation'].apply(lambda x: ', '.join(x))

            # 토픽 ID와 키워드를 원본 데이터프레임에 연결
            df_with_topics = df.copy()
            df_with_topics['topic_id'] = topics
            df_with_topics['topic_keywords'] = df_with_topics['topic_id'].map(
                dict(zip(topic_df['Topic'], topic_df['Representation']))
            )

            # 주 단위 시간 생성 (시작일은 월요일)
            df_with_topics['time_unit'] = df_with_topics['inp_date'].dt.to_period('W').apply(lambda r: r.start_time)

            # 주 단위로 토픽 카운트 집계
            weekly_data = df_with_topics.groupby(['time_unit', 'topic_id']).size().reset_index(name='count')
        else:
            # 사전 집계된 데이터 사용
            weekly_data = custom_data.reset_index()

        # 각 토픽별 trace 추가
        for i, topic_id in enumerate(all_topics):
            topic_data = weekly_data[weekly_data['topic_id'] == topic_id] if 'topic_id' in weekly_data.columns else None

            if topic_data is not None and not topic_data.empty:
                # 토픽 단어 가져오기
                topic_words = [word for word, _ in topic_model.get_topic(topic_id)][:3]
                topic_label = f"토픽 {topic_id}: {', '.join(topic_words)}"

                fig.add_trace(go.Scatter(
                    x=topic_data['time_unit'],
                    y=topic_data['count'],
                    mode='lines+markers',
                    name=topic_label,
                    line=dict(color=colors[i % len(colors)]),
                    marker=dict(size=8),
                    hovertemplate='<b>%{text}</b><br>주 시작일: %{x|%Y-%m-%d}<br>문서 수: %{y}<extra></extra>',
                    text=[topic_label] * len(topic_data)
                ))

        # 레이아웃 설정
        fig.update_layout(
            title={
                'text': '삼성전자 뉴스 키워드 분석<br><sup>총기간 2025-01-12 ~ 2025-04-03 : 총 12주 주말 제외, 월 ~ 금 주단위</sup>',
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {'size': 20, 'color': '#1f1f1f'}
            },
            xaxis=dict(title='주 시작일', tickformat='%Y-%m-%d', gridcolor='lightgray'),
            yaxis=dict(title='문서 수', gridcolor='lightgray'),
            legend=dict(title='토픽', orientation='v'),
            hovermode='closest',
            plot_bgcolor='white'
        )

    else:
        raise ValueError("지원되지 않는 시간 단위입니다. 'day' 또는 'week'를 사용하세요.")

    return fig

In [ ]:
# 일별 시계열 그래프
daily_fig = create_topic_timeseries(df, topics, topic_model, time_unit='day')
daily_fig.show()
print()

# 주별 시계열 그래프
weekly_fig = create_topic_timeseries(df, topics, topic_model, time_unit='week')
weekly_fig.show()

# 문서 특징

In [ ]:
import pandas as pd
import plotly.express as px

# 🗓️ 날짜만 추출 (시간 제거)
df['date_only'] = df['inp_date'].dt.date

# 📊 날짜별 문서 수 집계
daily_counts = df.groupby('date_only').size().reset_index(name='문서수')

# 📈 선 그래프 시각화
fig = px.line(daily_counts, x='date_only', y='문서수',
              title='📅 날짜별 문서 수',
              labels={'date_only': '날짜', '문서수': '문서 수'},
              markers=True)

fig.update_layout(
    xaxis_title='날짜',
    yaxis_title='문서 수',
    hovermode='x unified'
)

fig.show()

import pandas as pd
import plotly.express as px

# ⏱️ 기준 시작일 (데이터 중 가장 오래된 날짜)
start_date = df['inp_date'].min().normalize()

# ⌛ 며칠 지났는지 계산 → 7일 단위로 그룹 번호 지정
df['custom_week_group'] = ((df['inp_date'] - start_date).dt.days // 7)

# 📅 그룹의 시작 날짜 컬럼 생성 (optional)
df['custom_week_start'] = df['custom_week_group'].apply(lambda x: start_date + pd.Timedelta(days=7 * x))

# 📊 그룹별 문서 수 집계
weekly_custom = df.groupby('custom_week_start').size().reset_index(name='문서수')

# 📈 시각화
fig = px.line(weekly_custom, x='custom_week_start', y='문서수',
              title='🗓️ 사용자 정의 기준 주간 문서 수 (처음 날짜 기준)',
              labels={'custom_week_start': '주 시작일', '문서수': '문서 수'},
              markers=True)

fig.update_layout(
    xaxis_title='주차 (사용자 기준)',
    yaxis_title='문서 수',
    hovermode='x unified'
)

fig.show()

In [ ]:
barchart = topic_model.visualize_barchart()
barchart.show()
print('\n')

heatmap = topic_model.visualize_heatmap()
heatmap.show()
print('\n')

hierarchy = topic_model.visualize_hierarchy()
hierarchy.show()
print('\n')

term_rank = topic_model.visualize_term_rank()
term_rank.show()
print('\n')

topics = topic_model.visualize_topics()
topics.show()

### Q) 상승하고 있다고 해도 다른 토픽의 최하점에도 못미치는 경우에는 정말 상승한다고 볼 수 있을것인가?

✅ 방법 1: 상대 상승률 + 절대값 조건 함께 사용
growth_df = growth_df[(growth_df['Recent Avg'] > 10)]  # 예: 최근 평균 문서 수 10 이상
→ 즉, 상승한 건 맞지만 어느 정도 규모는 있어야 진짜 급상승으로 인정

✅ 방법 2: Z-score 기반 이상치 탐지
전체 토픽의 최근 평균 분포를 보고,

평균 대비 얼마나 벗어난 정도인지(표준편차 기준)를 보정하는 방식
from scipy.stats import zscore

growth_df['Z-score'] = zscore(growth_df['Increase'])
growth_df = growth_df[growth_df['Z-score'] > 1.5]

✅ 방법 3: "상대순위 변화" 기반 접근
이전 주차에 비해 현재 전체 토픽 중 순위가 얼마나 올랐는가를 보는 방법

예:

갤럭시가 전체 순위 23위 → 최근 주 5위 → 18단계 상승
→ 이런 방식은 작은 토픽도 고려하되, 무시할 수준의 작음은 제외 가능

In [ ]:
#수정필요 실행안해봄
def get_trending_topics(df, n_weeks=3, top_k=5, min_recent_avg=10, min_increase_pct=30, sort_by='increase'):
    """
    최근 n주 동안 급상승한 토픽을 자동으로 리포트해주는 함수

    Parameters:
    - df: 데이터프레임 (주 단위 컬럼 'inp_date', 'topic_keywords' 필요)
    - n_weeks: 최근 비교할 주 수
    - top_k: 리포트할 토픽 수
    - min_recent_avg: 최근 평균 문서 수 최소 기준
    - min_increase_pct: 최소 증가 비율 %
    - sort_by: 정렬 기준 ('increase' or 'increase_pct')

    Returns:
    - 상승 토픽 요약 DataFrame
    """
    import pandas as pd

    df['week'] = df['inp_date'].dt.to_period('W').apply(lambda r: r.start_time)
    weekly_counts = df.groupby(['week', 'topic_keywords']).size().unstack(fill_value=0)

    recent_weeks = weekly_counts.index[-n_weeks:]
    prev_weeks = weekly_counts.index[-2*n_weeks:-n_weeks]

    recent_avg = weekly_counts.loc[recent_weeks].mean()
    prev_avg = weekly_counts.loc[prev_weeks].mean()

    increase = recent_avg - prev_avg
    increase_pct = ((increase) / (prev_avg + 1e-6)) * 100

    growth_df = pd.DataFrame({
        'Prev Avg': prev_avg,
        'Recent Avg': recent_avg,
        'Increase': increase,
        'Increase (%)': increase_pct
    })

    # 조건 필터링
    growth_df = growth_df[
        (growth_df['Recent Avg'] >= min_recent_avg) &
        (growth_df['Increase (%)'] >= min_increase_pct)
    ]

    # 정렬
    if sort_by == 'increase_pct':
        growth_df = growth_df.sort_values(by='Increase (%)', ascending=False)
    else:
        growth_df = growth_df.sort_values(by='Increase', ascending=False)

    return growth_df.head(top_k).round(2)

In [ ]:
trending = get_trending_topics(df,
                                n_weeks=3,
                                top_k=5,
                                min_recent_avg=10,
                                min_increase_pct=50,
                                sort_by='increase_pct')

print("🔥 최근 3주간 급상승한 토픽 TOP 5")
print(trending)

### Q) 동일한 내용의 뉴스가 동시다발적으로 올려진 경우(동일한 뉴스를 뿌린경우)가 존재하는가?

In [ ]:
# 'content' 컬럼에서 중복된 값들만 추출 (모든 중복 포함)
duplicated_rows = df[df.duplicated(subset='content', keep=False)]

# 결과 출력
print(duplicated_rows[['content']])


In [ ]:
# 중복된 content만 필터링 (모든 중복 포함)
duplicated_rows = df[df.duplicated(subset='content', keep=False)]

# 중복 문서 중에서 고유한 것만 한 번씩만 남기기
unique_duplicates = duplicated_rows.drop_duplicates(subset='content')

# 결과 출력
print(unique_duplicates[['content']])


### Q) 데이터 수집 방식의 차이, 기자가 살짝만 수정한 것으로 인해 거의 유사함에도 중복데이터로 처리되지 않은 경우는 없을까?

In [ ]:
## 너무 느려서 faiss로 코드 재구축 필요

from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
from tqdm import tqdm

# 1. 임베딩 모델 준비
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 2. 문서 리스트
docs = df['content'].tolist()

# 3. 임베딩 생성
embeddings = model.encode(docs, convert_to_tensor=True, show_progress_bar=True)

# 4. 유사도 행렬 계산
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)

# 5. 유사도 0.95 이상인 문서 쌍을 기반으로 중복 그룹 만들기
threshold = 0.95
visited = set()
duplicate_groups = []

for i in tqdm(range(len(docs))):
    if i in visited:
        continue
    group = [i]
    for j in range(i + 1, len(docs)):
        if cosine_scores[i][j] >= threshold:
            group.append(j)
            visited.add(j)
    if len(group) > 1:
        duplicate_groups.append(group)
        visited.update(group)

# 6. 중복 그룹 확인
print(f"유사한 문서 그룹 수: {len(duplicate_groups)}")

# 7. 각 그룹에서 하나만 남기고 나머지를 제거 대상으로 기록
to_remove = set()
for group in duplicate_groups:
    # group[0]만 남기고 나머지 제거
    to_remove.update(group[1:])

# 8. 중복된 문서만 따로 추출
semantic_duplicates = df.iloc[list(to_remove)]

# 9. 중복을 제외한 고유한 문서만 추출
df_semantic_dedup = df.drop(index=to_remove).reset_index(drop=True)

# 10. 결과 출력
print(f"중복 문서 수 (유사도 {threshold} 이상): {len(to_remove)}")
print("✅ 고유한 문서 수:", df_semantic_dedup.shape[0])


Q) 벡터라이저 종류 다르게?
Q) UMAP 차원 축소를 몇개로 해야 적합할지?
Q) HDBSCAN 최소 클러스터 크기를 몇개로 하는 것이 좋을지?
Q) 문장 임베딩 모델을 다르게?
Q) 사용자 사전 구축이 필요할지? 필요하다면 어떻게 구축할지?